In [ ]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

# 儲存PTT Stock文章列表每頁的網址
ptt_url = []
# https://www.ptt.cc/bbs/Stock/index4997.html(最新的上頁)
pages = 4997

for i in range(1,pages):
    res = requests.get(f'https://www.ptt.cc/bbs/Stock/index{i}.html')
    soup = BeautifulSoup(res.text, 'html.parser')
    a_s = soup.select('div.title > a')
    for a in a_s:
        try:
            url = a.get('href')
            ptt_url.append('https://www.ptt.cc'+url)
        except:
            print('該篇文章已刪除')

# 存取網頁內容
author_store = [] 
title_store = []
date_store = []
articles_store = []
url_list = []
record = 0
for g in ptt_url:
    try:
        ptt_all = requests.get(g)   # 取得所有網址內容
        soup_2 = BeautifulSoup(ptt_all.text,'html.parser') # 解析
    except:
        continue
    
    try:
        z = soup_2.find(id="main-content")
        metas = z.select('div.article-metaline')
        author = metas[0].select('span.article-meta-value')[0].text
        title = metas[1].select('span.article-meta-value')[0].text 
        date = metas[2].select('span.article-meta-value')[0].text
        articles = soup_2.select('#main-content')[0].text
        
        url_list.append(g)
        author_store.append(author)
        title_store.append(title)
        date_store.append(date)

        articles = articles.replace(author,'')
        articles = articles.replace(title,'')
        articles = articles.replace(date,'')
        articles = articles.replace('作者看板stock標題時間','')
        articles = articles.replace(" ", "")
        articles = articles.replace('\n', "")
       
        articles_store.append(articles)
        record+=1
        print('處理了:',(record/len(ptt_url))*100,'%')
    except:
        continue


        
         

pd_store= pd.DataFrame(columns = ['Author','Title','Date','Content','URL'])
pd_store['Author'] = author_store
pd_store['Title'] = title_store
pd_store['Date'] = date_store
pd_store['Content'] = articles_store
pd_store['URL'] = url_list


pd_store.to_csv('ptt_test.csv',index=False)